# API development for download of Sentinel-2 and Landsat-8 data
## User defined mosaicking on harmonised products

Two different multi-spectral optical satellite constellations deliver open data source for earth observation.
The Sentinel-2 mission contains two identical satellites with suffix A and B, with opposite direction of orbits flying in a 
repeat cycle of 10 days each, 5 days for both orbits, respectively. Launch: 2015 (A), 2017 (B)[ESA].
<br>
As the second optical satellite Landsat 8 Operational Land Imager (OLI),...
with a repeat cycle of 16 days.  Launch: 2013, come with [res. etc.]... 2013
<br>


The scope of this extension to Ben Mack's [email/github] aims to add the following to the program:
1. 


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import urllib
import geopandas as gp
from nasa_hls.utils import get_available_datasets
from nasa_hls.download_hls_dataset import download_batch
import pandas as pd